TESTING MODEL & ASSESSING ACCURACY
- Test model on individual discrete patients and on outliers
- Generate probability colour maps
- Calculate overlap metrics

In [2]:
import SimpleITK as sitk
import pandas as pd
import glob
import numpy as np
import os
import joblib

Create DataFrames for Test Patients (Non-outliers and outliers)

In [3]:
# load in model
p_clf = joblib.load("p-rbf-clf.joblib")

In [4]:
# function to create the DataFrame columns
def create_test_col(dir, p_ID, mode):
      # modalities =  GM, MD, FA, FICVF, ODI
    modes = {'GM': dir + 'GM-OUTPUT/GM-ZMAPS/',
            'MD': dir + 'MD-OUTPUT/MD-ZMAPS/',
            'FA': dir + 'FA-OUTPUT/FA-ZMAPS/',
            'FICVF': dir + 'FICVF-OUTPUT/FICVF-ZMAPS/',
            'ODI': dir + 'ODI-OUTPUT/ODI-ZMAPS/'
           }
    # data is sitk image type
    # p_ID is patient ID (string)
    
    file = glob.glob( modes[mode] + "*" + p_ID + "*.nii")
    if not file:
        print(mode + " modality doesn't exist for " + p_ID)
        return [0] * 2122945 # voxels in flattened mri

    img = sitk.ReadImage(file[0])
    arr = sitk.GetArrayFromImage(img)
    arr = arr.flatten()
    return arr

# function to bring DataFrame columns together into single DF
def create_test_df(dir, test_patient):
    data = {'GM' : create_test_col(dir, test_patient, "GM"),
                        'FA' : create_test_col(dir, test_patient, "FA"),
                        'MD' : create_test_col(dir, test_patient, "MD"),
                        'FICVF' : create_test_col(dir, test_patient, "FICVF"),
                        'ODI' : create_test_col(dir, test_patient, "ODI"),
        }
    df = pd.DataFrame(data, index=None)
    return df

In [5]:
# these are the patients we are testing the model with and generation probability maps on
# patients have not been seen by the model during training process
patient_ids = ["D046", "D047", "D048", 'D049']
patient_dfs = []
dir = "OUTPUT/DISCRETE/"

print("Creating test patient DFS...")
for patient in patient_ids:
    df = create_test_df(dir, patient)
    patient_dfs.append(df)
    print("DF created for patient: ", patient)

# outlier patients have larger lesion sizes than training data used (~2500 voxels)
# D021 lesion size: 6645 voxels
# D035 lesion size: 11836 voxels
outlier_ids = ["D021", "D035"]
outlier_dfs = []
print("\nCreating outlier patient DFS...")
for outlier in outlier_ids:
    df = create_test_df(dir, outlier)
    outlier_dfs.append(df)
    print("DF created for outlier: ", outlier)
    
print("Patient & Outlier DataFrames Created")

Creating test patient DFS...
DF created for patient:  D046
DF created for patient:  D047
DF created for patient:  D048
DF created for patient:  D049

Creating outlier patient DFS...
DF created for outlier:  D021
DF created for outlier:  D035
Patient & Outlier DataFrames Created


Generate Probability Colour Maps

In [5]:
# Function to generate probability maps

# simply using example_img so we can use the sitk function CopyInformation to ensure when we convert from
# array to image, the original image properties are maintained
example_img = sitk.ReadImage("OUTPUT/DISCRETE/GM-OUTPUT/GM-ZMAPS/Z-smwc1D049_T1.nii") 
def generate_colour_map(OUTPUT_DIR, df, p_ID):
    X_test_patient = df
    print("Predicting probability of patient: ", p_ID)
    y_pred_patient = p_clf.predict_proba(X_test_patient)
    print("Completed prediction of patient: ", p_ID)
    print("Converting to MRI")
    vals = [y[1] for y in y_pred_patient] # take probabilities that there is a lesion present
    vals = np.asarray(vals) 
    y_pred_patient_3D = vals.reshape((121, 145, 121)).transpose() # convert back to 3D array (MRI)
    y_pred_patient_3D_img = sitk.GetImageFromArray(y_pred_patient_3D) # convert 3D array to nifti image
    y_pred_patient_3D_img.CopyInformation(example_img)
    filename = "Probability-Map-" + p_ID + ".nii"
    print("Writing image...")
    sitk.WriteImage(y_pred_patient_3D_img, os.path.join(OUTPUT_DIR, filename))
    print("Probability map generated for patient: ", p_ID)
    
    
    

In [6]:
print("Generating probability maps for test patients...")
dir ="OUTPUT/PROBABILITY-MAPS/TEST-PATIENTS"

for df, p_id in zip(patient_dfs, patient_ids):
    generate_colour_map(dir, df, p_id)

Generating probability maps for test patients...
Predicting probability of patient:  D046
Completed prediction of patient:  D046
Converting to MRI
Writing image...
Probability map generated for patient:  D046
Predicting probability of patient:  D047
Completed prediction of patient:  D047
Converting to MRI
Writing image...
Probability map generated for patient:  D047
Predicting probability of patient:  D048
Completed prediction of patient:  D048
Converting to MRI
Writing image...
Probability map generated for patient:  D048
Predicting probability of patient:  D049
Completed prediction of patient:  D049
Converting to MRI
Writing image...
Probability map generated for patient:  D049


In [7]:
print("Generating probability maps for outliers...")
dir ="OUTPUT/PROBABILITY-MAPS/OUTLIERS"

for df, p_id in zip(outlier_dfs, outlier_ids):
    generate_colour_map(dir, df, p_id)

Generating probability maps for outliers...
Predicting probability of patient:  D021
Completed prediction of patient:  D021
Converting to MRI
Writing image...
Probability map generated for patient:  D021
Predicting probability of patient:  D035
Completed prediction of patient:  D035
Converting to MRI
Writing image...
Probability map generated for patient:  D035


Segment Probability Maps

In [18]:
# we want to exclude the air around the resulting probability map so we use
# a segmentation label created using 3DSlicer and simply multiply the 2 images together
def segment_prob_map(dir, predicted_img, segment_mask_arr):
    pred_img = sitk.ReadImage(predicted_img)
    pred_img_arr = sitk.GetArrayFromImage(pred_img)
    segmented_pred = segment_mask_arr * pred_img_arr
    segmented_pred_img = sitk.GetImageFromArray(segmented_pred)
    segmented_pred_img.CopyInformation(example_img)
    filename = "S-" + os.path.basename(predicted_img) + ".nii"
    sitk.WriteImage(segmented_pred_img, os.path.join(dir, filename))
    
# 
# segment_mask = sitk.ReadImage('Segmentation-label.nrrd')
# segment_mask_arr = sitk.GetArrayFromImage(segment_mask)
# segment_mask_arr[segment_mask_arr == 2] = 0
# 
# predicted_img_arr = sitk.GetArrayFromImage(y_pred_patient_3D_img)
# segmented_pred= segment_mask_arr * predicted_img_arr
# segmented_pred_img = sitk.GetImageFromArray(segmented_pred)
# segmented_pred_img.CopyInformation(gm_img)
# OUTPUT_DIR = "OUTPUT/PROBABILITY-MAPS/SEGMENTED-PROB-MAPS"
# filename = "S-P-" + os.path.basename(gm_zmap[0])
# sitk.WriteImage(segmented_pred_img, os.path.join(OUTPUT_DIR, filename))

In [9]:
segment_mask = sitk.ReadImage("Segmentation-label.nrrd")
mask_arr = sitk.GetArrayFromImage(segment_mask)
mask_arr[mask_arr == 2] = 0 # ensure that positive labels are 1 and negative labels are 0 

In [22]:
print("Segmenting probability maps for test patients...")
test_maps = glob.glob("OUTPUT/PROBABILITY-MAPS/TEST-PATIENTS/*.nii")
dir ="OUTPUT/PROBABILITY-MAPS/TEST-PATIENTS/SEGMENTED-MAPS"
for map in test_maps:
    segment_prob_map(dir, map, mask_arr)
print("Segmented probability maps created for test patients")

Segmenting probability maps for test patients...
Segmented probability maps created for test patients


In [21]:
print("Segmenting probability maps for outliers...")
dir ="OUTPUT/PROBABILITY-MAPS/OUTLIERS/SEGMENTED-MAPS"
outlier_maps = glob.glob("OUTPUT/PROBABILITY-MAPS/OUTLIERS/*.nii")
for map in outlier_maps:
    segment_prob_map(dir, map, mask_arr)
print("Outlier probability maps segmented")

Segmenting probability maps for outliers...
Outlier probability maps segmented


Calculate Dice overlap for one patient

In [6]:
# import lesion segmentation map to isolate smaller area around abnormal voxel prediction
# in order to more accurately predict dice score
lesion_seg_area = sitk.ReadImage("Lesion-Seg-D048.nrrd")
lesion_seg_area = sitk.GetArrayFromImage(lesion_seg_area)
lesion_seg_area[lesion_seg_area == 2 ] = 0 # ensure positive labels are 1 and negative labels are 0

In [24]:
# segment area around probability map where abnormal voxels were predicted near actual lesion
patient_48_prob = sitk.ReadImage("OUTPUT/PROBABILITY-MAPS/TEST-PATIENTS/SEGMENTED-MAPS/S-Probability-Map-D048.nii")
patient_48_arr = sitk.GetArrayFromImage(patient_48_prob)
# in order to calculate dice coefficient we must convert probability to binary
patient_48_arr[patient_48_arr >= 0.5 ] = 1
patient_48_arr[patient_48_arr < 0.5] = 0
seg_patient_48_arr = patient_48_arr * lesion_seg_area # segment area
patient_48_lesion_arr = sitk.GetArrayFromImage(sitk.ReadImage("DISCRETE/Lesion-Masks/wD048_Lesion.nii"))

seg = seg_patient_48_arr
truth = patient_48_arr

k = 1
dice = np.sum(seg[truth==k]==k)*2.0 / (np.sum(seg[seg==k]==k) + np.sum(truth[truth==k]==k))

print("Dice similarity score is {}".format(dice))


Dice similarity score is 0.07958726081372314
